In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))
# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'gender_submission.csv']


Lets start by getting the data into a dataframe

In [2]:
# test dataframe
df = pd.read_csv('./input/test.csv')
# I prefer sampling to head since the randomization for the rows we get can be valuable 
# and also because it seems to fill in screen better
df.sample(6)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
100,992,1,"Stengel, Mrs. Charles Emil Henry (Annie May Mo...",female,43.0,1,0,11778,55.4417,C116,C
289,1181,3,"Ford, Mr. Arthur",male,NaN,0,0,A/5 1478,8.0500,NaN,S
82,974,1,"Case, Mr. Howard Brown",male,49.0,0,0,19924,26.0000,NaN,S
293,1185,1,"Dodge, Dr. Washington",male,53.0,1,1,33638,81.8583,A34,S
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
288,1180,3,"Mardirosian, Mr. Sarkis",male,NaN,0,0,2655,7.2292,F E46,C


Looking for nan values in the data.
We must clean these in order to be able to work with the dataset. 

In [3]:
df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

We have age and cabin issues. 

For the age, ideas are may be:
1. Fill it with the average of ages
2. Check wether parents age and imagine they should not be less than 20 years younger than their offspring
3. If they do have parents, look for siblings and check their age. Maybe the gap isn't that big.

Key: 
- SibSp - siblings/spouse  - This means theres someone else that is probably boarded on the same pier, around the same age and potentially staying in the same cabin
- Parch - parents /children - This may mean theres someone about 16-18 years older/younger on board, that may have boarded on the same pier and potentially staying in the same cabin.

Both of the hipothesis above should be tested for variance.




In [4]:
# this is solution 1
fixed_age = df['Age'].fillna(df.Age.mean)
df.Age = fixed_age
df.isnull().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
# show the head of rows where the age is not nan to confirm we still have a valid dataset
df[(df.Age.notnull())].head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,NaN,S


In [6]:
# Show whats left
df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
# What exactly is this number representing?
num_cabins = df.PassengerId.count()
num_nans = df.Cabin.isnull().sum()
num_filled = df.Cabin.notnull().sum()
print("% of NaN's:",  num_nans / num_cabins)
print("% of correct cabin numbers:", num_filled / num_cabins)

% of NaN's: 0.7822966507177034
% of correct cabin numbers: 0.21770334928229665


With only 20% for the cabin numbers inserted, it would be very hard to 
The solution for the missing cabins is:
1. set the missing ones to cabin -1 so we know that cabin cannot be used but its not missing
2. Find out the cabins placement via the ticket type and by looking at the ship blueprint - Then we can assume also that the cabin location may have have weight in the individual survivability. 


In [8]:
# Trying solution #1 and making some exploratory code
df.Cabin = df['Cabin'].fillna(-1)
known_cabins = df.Cabin.unique()
# count all but "-1" cabins which will be at the top of the list
len(known_cabins[1:])

76

76 cabins and 418 passengers. 
[The encyclopedia titanica](https://www.encyclopedia-titanica.org/cabins.html) also presents some interesting numbers, but as far we are concerned for the matter of survivability this is not a defining factor at the moment.
We will revisit this if we think this is necessary.